In [12]:
# first let's do all of the import statements
import requests
import urllib2
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from collections import defaultdict
from string import punctuation
from heapq import nlargest
from math import log
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans



In [13]:
def getWashPostText(url,token):
    # THis function takes the URL of an article in the 
    # Washington Post, and then returns the article minus all 
    # of the crud - HTML, javascript etc. How? By searching for
    # everything that lies between the tags titled 'token'
    # Like most web-scraping, this will only work for urls where
    # we know the structure (eg say all articles in the WashPo are
    # enclosed in <article></article> tags). This will also change from
    # time to time as different HTML formats are employed in the website
    try:
       # page = urllib2.urlopen(url).read().decode('utf8')
        page = requests.get(url,verify=False)
    except:
        # if unable to download the URL, return title = None, article = None
        return (None,None)
    soup = BeautifulSoup(page)
    if soup is None:
        return (None,None)
    # If we are here, it means the error checks were successful, we were
    # able to parse the page
    text = ""
    if soup.find_all(token) is not None:
        # Search the page for whatever token demarcates the article
        # usually '<article></article>'
        text = ''.join(map(lambda p: p.text, soup.find_all(token)))
        # mush together all the text in the <article></article> tags
        soup2 = BeautifulSoup(text)
        # create a soup of the text within the <article> tags
        if soup2.find_all('p')!=[]:
            # now mush together the contents of what is in <p> </p> tags
            # within the <article>
            text = ''.join(map(lambda p: p.text, soup2.find_all('p')))
    return text, soup.title.text
    # what did we just do? Let's go through and understand
    # finally return the result tuple with the title and the body of the article
    



In [14]:

# Now we will do something very very similar, but this time for the New York Times
def getNYTText(url,token):
    response = requests.get(url)
    # THis is an alternative way to get the contents of a URL
    soup = BeautifulSoup(response.content,"lxml")
    page = str(soup)
    title = soup.find('title').text
    mydivs = soup.findAll("p", {"class":"story-body-text story-content"})
    text = ''.join(map(lambda p:p.text, mydivs))
    return text, title
    # Notice again how important it is to know the structure of the page
    # we are seeking to scrape. If we did not know that articles in the NYT
    # come contained in these tags - an outer tag <p> and an inner tag
    # of class = story-body-text story-content, we would be unable to parse


In [15]:

# Ok! Now we have a way to extract the contents and title of an individual
# URL. Let's hook this up inside another function that will take the URL
# of an entire section of a newspaper - say the Technology or Sports section
# of a newspaper - and parse all of the URLs for articles linked off that
# section. 
# Btw, these sections also come with plenty of non-news links - 'about',
# how to syndicate etc, so we will employ a little hack - we will consider
# something to be a news article only if the url has a dateline. THis is 
# actually very safe - its pretty much the rule for articles to have a 
# date, and virtually all important newspapers mush this date into the URL.
def scrapeSource(url, magicFrag='2015',scraperFunction=getNYTText,token='None'):
    urlBodies = {}
    request = urllib2.Request(url)
    response = urllib2.urlopen(request)
    soup = BeautifulSoup(response)
    # we are set up with a Soup of the page - now find the links
    # Remember that links are always of the form 
    # <a href='link-url'> link-text </a>
    numErrors = 0
    for a in soup.findAll('a'):
        try:
            url = a['href']
            if( (url not in urlBodies) and 
               ((magicFrag is not None and magicFrag in url) 
               or magicFrag is None)):
                body = scraperFunction(url,token)
                # this line above is important - scraperFunction 
                # refers to the individual scraper function for the 
                # new york times or the washington post etc.
                if body and len(body) > 0:
                    urlBodies[url] = body
                print url
        except:
            numErrors += 1
            # plenty of parse errors happen - links might not
            # be external links, might be malformed and so on -
            # so don't mind if there are exceptions.
    return urlBodies


In [ ]:

# Now for the frequency summarizer class - which we have encountered
# before. To quickly jog our memories - given an (title,article-body) tuple
# the frequency summarizer has easy ways to find the most 'important'
# sentences, and the most important words. How is 'important' defined?
# Important = most frequent, excluding 'stopwords' which are generic
# words like 'the' etc which can be ignored
class FrequencySummarizer:
    def __init__(self,min_cut=0.1,max_cut=0.9):
        # class constructor - takes in min and max cutoffs for 
        # frequency
        self._min_cut = min_cut
        self._max_cut = max_cut
        self._stopwords = set(stopwords.words('english') +
                              list(punctuation) +
                              [u"'s",'"'])
        # notice how the stopwords are a set, not a list. 
        # its easy to go from set to list and vice-versa
        # (simply use the set() and list() functions) - 
        # but conceptually sets are different from lists
        # because sets don't have an order to their elements
        # while lists do
    
    def _compute_frequencies(self,word_sent,customStopWords=None):
        freq = defaultdict(int)
        # we have encountered defaultdict objects before
        if customStopWords is None:
            stopwords = set(self._stopwords)
        else:
            stopwords = set(customStopWords).union(self._stopwords)
        for sentence in word_sent:
            for word in sentence:
                if word not in stopwords:
                    freq[word] += 1
        m = float(max(freq.values()))
        for word in freq.keys():
            freq[word] = freq[word]/m
            if freq[word] >= self._max_cut or freq[word] <= self._min_cut:
                del freq[word]
        return freq
    
    def extractFeatures(self,article,n,customStopWords=None):
        # The article is passed in as a tuple (text, title)
        text = article[0]
        # extract the text
        title = article[1]
        # extract the title
        sentences = sent_tokenize(text)
        # split the text into sentences
        word_sent = [word_tokenize(s.lower()) for s in sentences]
        # split the sentences into words 
        self._freq = self._compute_frequencies(word_sent,customStopWords)
        # calculate the word frequencies using the member function above
        if n < 0:
            # how many features (words) to return? IF the user has
            # asked for a negative number, this is a sign that we don't
            # do any feature selection - we return ALL features
            # THis is feature extraction without any pruning, ie no
            # feature selection (beyond simply picking words as the features)
            return nlargest(len(self._freq_keys()),self._freq,key=self._freq.get)
        else:
            # if the calling function has asked for a subset then
            # return only the 'n' largest features - ie here the most
            # important words (important == frequent, barring stopwords)
            return nlargest(n,self._freq,key=self._freq.get)
        # let's summarize what we did here. 
    
    def extractRawFrequencies(self, article):
        # very similar, except that this method will return the 'raw'
        # frequencies - literally just the word counts
        text = article[0]
        title = article[1]
        sentences = sent_tokenize(text)
        word_sent = [word_tokenize(s.lower()) for s in sentences]
        freq = defaultdict(int)
        for s in word_sent:
            for word in s:
                if word not in self._stopwords:
                    freq[word] += 1
        return freq
    
    def summarize(self, article,n):
        text = article[0]
        title = article[1]
        sentences = sent_tokenize(text)
        word_sent = [word_tokenize(s.lower()) for s in sentences]
        self._freq = self._compute_frequencies(word_sent)
        ranking = defaultdict(int)
        for i,sentence in enumerate(word_sent):
            for word in sentence:
                if word in self._freq:
                    ranking[i] += self._freq[word]
        sentences_index = nlargest(n,ranking,key=ranking.get)

        return [sentences[j] for j in sentences_index]


In [11]:

urlWashingtonPostNonTech = "https://www.washingtonpost.com/sports"
urlNewYorkTimesNonTech = "https://www.nytimes.com/pages/sports/index.html"
urlWashingtonPostTech = "https://www.washingtonpost.com/business/technology"
urlNewYorkTimesTech = "http://www.nytimes.com/pages/technology/index.html"

washingtonPostTechArticles = scrapeSource(urlWashingtonPostTech,
                                          '2016',
                                         getWashPostText,
                                         'article') 
washingtonPostNonTechArticles = scrapeSource(urlWashingtonPostNonTech,
                                          '2016',
                                         getWashPostText,
                                         'article')
                
                
newYorkTimesTechArticles = scrapeSource(urlNewYorkTimesTech,
                                       '2016',
                                       getNYTText,
                                       None)
newYorkTimesNonTechArticles = scrapeSource(urlNewYorkTimesNonTech,
                                       '2016',
                                       getNYTText,
                                       None)


HTTPError: HTTP Error 403: Forbidden